# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [ ]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [ ]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [ ]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-10-11'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-EUR',start_date, today).reset_index()

btc_df

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-01-01,$396.76,$401.77,$393.73,$400.01,$400.01,33412139
1,2016-01-02,$400.28,$401.60,$397.74,$399.19,$399.19,27718364
2,2016-01-03,$399.32,$399.47,$391.15,$396.36,$396.36,36532215
3,2016-01-04,$396.41,$401.62,$395.80,$400.19,$400.19,35554888
4,2016-01-05,$400.17,$401.31,$399.88,$401.90,$401.90,32120022
...,...,...,...,...,...,...,...
2835,2023-10-06,"$26,457.89","$26,689.29","$25,988.70","$25,995.52","$25,995.52",11261905035
2836,2023-10-07,"$25,991.93","$26,668.98","$25,928.07","$26,380.19","$26,380.19",12736143050
2837,2023-10-08,"$26,380.37","$26,457.12","$26,308.29","$26,401.19","$26,401.19",6185746183
2838,2023-10-09,"$26,403.87","$26,527.04","$26,166.60","$26,437.54","$26,437.54",7492467439


In [ ]:
# TAREA: Validamos que no hay datos vacíos
btc_df.isnull().sum()


Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[["Date","Open"]]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
new_names = {
    "Date": "ds",
    "Open": "y",
}

# Los renombramos con los nuevos nombres
df.rename(columns=new_names, inplace=True)
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
2835,2023-10-06,"$26,457.89"
2836,2023-10-07,"$25,991.93"
2837,2023-10-08,"$26,380.37"
2838,2023-10-09,"$26,403.87"
2839,2023-10-10,"$26,436.96"


In [ ]:
# Crear la gráfica del precio de apertura

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título
fig.update_layout(
    title_text="Serie temporal de Bitcoin Precio de Apertura",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



Documentacion oficial de Prophet [GitHub](https://facebook.github.io/prophet/docs/quick_start.html)

In [ ]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
from prophet import Prophet
m = Prophet(seasonality_mode='multiplicative')

In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpa9ai2utt/r5r2uabv.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpa9ai2utt/tutsoosp.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=54656', 'data', 'file=/tmp/tmpa9ai2utt/r5r2uabv.json', 'init=/tmp/tmpa9ai2utt/tutsoosp.json', 'output', 'file=/tmp/tmpa9ai2utt/prophet_modelv8fpss2z/prophet_model-20231011100205.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:02:05 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:02:06 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [ ]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe() - genera las fechas en futuro
future = m.make_future_dataframe(periods=366)
future.tail()

,ds
3201,2024-10-06
3202,2024-10-07
3203,2024-10-08
3204,2024-10-09
3205,2024-10-10


In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
3,2016-01-04
4,2016-01-05
5,2016-01-06
6,2016-01-07
...,...
3199,2024-10-04
3202,2024-10-07
3203,2024-10-08
3204,2024-10-09


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [ ]:
#Utiliza el modelo m para predecir el dataframe future
forecast =  m.predict(future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2285,2024-10-04,"$15,240.50",$520.34,"$29,307.72"
2286,2024-10-07,"$15,664.71",$-609.32,"$30,519.60"
2287,2024-10-08,"$15,820.73",$-109.71,"$31,289.02"
2288,2024-10-09,"$15,921.40",$-749.04,"$30,686.29"
2289,2024-10-10,"$16,133.69",$-734.39,"$31,368.52"


In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

20469.25569385902

<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [ ]:
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)